### location file format : map-making-app -> GeoGuessr

In [1]:
import json
from geopy.geocoders import Nominatim

In [2]:
tag_lauage = 'jp'
# tag_lauage = 'en'

In [3]:
def is_target_format(data):
    """
    入力データがすでに変換後のフォーマットに一致しているかを確認。
    """
    if isinstance(data, list):  # データがリストであることを確認
        for item in data:
            if not isinstance(item, dict) or "lat" not in item or "lng" not in item:
                return False
        return True
    return False

In [4]:
# 緯度と経度から国名を返す
def get_country_name(latitude, longitude):
    geolocator = Nominatim(user_agent="abc")
    location = geolocator.reverse((latitude, longitude), language=tag_lauage)
    
    if location and 'country' in location.raw['address']:
        return location.raw['address']['country']
    else:
        return "unknown"

#### JSON構造修正して国名のタグ追加

In [14]:
# 入力ファイルと出力ファイルのパス
input_file = "./input/map-making-app_1.json"
output_file = "./output/geoguessr_4.json"

# JSONファイルを読み込んで、customCoordinatesを抽出し、必要なら変換後フォーマットに変更
try:
    with open(input_file, "r", encoding="utf-8") as infile:
        data = json.load(infile)

    if is_target_format(data):
        # 既に変換後のフォーマットの場合
        formatted_data = data
        print("入力データは既に変換フォーマットです。")
    else:
        # customCoordinatesを抽出
        custom_coordinates = data.get("customCoordinates", None)

        if custom_coordinates is None:
            print("customCoordinatesが見つかりませんでした。")
            formatted_data = []
        else:
            # customCoordinatesを出力データに設定
            formatted_data = custom_coordinates
            print("customCoordinatesを抽出しました。")

    cnt = 1
    # 各要素にextra属性を追加
    for item in formatted_data:
        if isinstance(item, dict) and "lat" in item and "lng" in item:
            country_name = get_country_name(float(item["lat"]), float(item["lng"]))

            if "extra" in item:
                # extraが既に存在する場合、tagsに"add"を追加
                if "tags" in item["extra"] and isinstance(item["extra"]["tags"], list):
                    item["extra"]["tags"].append(country_name)
                else:
                    # tagsが存在しない場合、新規作成
                    item["extra"]["tags"] = [country_name]
            else:
                # extraが存在しない場合、新規作成
                item["extra"] = {
                    "tags": [country_name],
                    "panoId": None,
                    "panoDate": None,
                }
        # 進捗を表示
        print(cnt , "/", len(formatted_data))
        cnt += 1

    # 結果を新しいJSONファイルに書き込む
    with open(output_file, "w", encoding="utf-8") as outfile:
        json.dump(formatted_data, outfile, ensure_ascii=False, indent=4)
    print(f"結果が {output_file} に保存されました。")

except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")

customCoordinatesを抽出しました。
1 / 4
2 / 4
3 / 4
4 / 4
結果が ./output/geoguessr_4.json に保存されました。


#### JSON形式だけ変換（タグ情報追加せず）

In [3]:
# 入力ファイルと出力ファイルのパス
input_file = './input/map-making-app_1.json'
output_file = './output/geoguessr_3.json'

# JSONファイルを読み込んで、customCoordinatesを抽出し、必要なら変換後フォーマットに変更
try:
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
    
    if is_target_format(data):
        # 既に変換後のフォーマットの場合
        output_data = data
        print("入力データは既に変換フォーマットです。")
    else:
        # customCoordinatesを抽出
        custom_coordinates = data.get('customCoordinates', None)
        
        if custom_coordinates is None:
            print("customCoordinatesが見つかりませんでした。")
            output_data = []
        else:
            # customCoordinatesを出力データに設定
            output_data = custom_coordinates
            print("customCoordinatesを抽出しました。")
    
    # 結果を新しいJSONファイルに書き込む
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(output_data, outfile, ensure_ascii=False, indent=4)
    print(f"結果が {output_file} に保存されました。")

except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")


customCoordinatesを抽出しました。
結果が ./output/geoguessr_3.json に保存されました。
